In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import csv
import requests
from bs4 import BeautifulSoup
import urllib2, urllib, json
import re
import datetime
import statsmodels.formula.api as smf # Regression library

!pip install unirest
import unirest

%matplotlib inline

You are using pip version 7.1.2, however version 8.1.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [78]:
dfNorthMotorwayInfo = pd.read_csv('../Datasets/Northern Motorway Road Segments.csv', sep='\t')
print dfNorthMotorwayInfo.info()
dfNorthMotorwayInfo.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 416 entries, 0 to 415
Data columns (total 1 columns):
id;full_name;road_segment_wkt    416 non-null object
dtypes: object(1)
memory usage: 6.5+ KB
None


,id;full_name;road_segment_wkt
0,606114469;Northern Mtwy;LINESTRING(174.76028 -...
1,606114470;Northern Mtwy;LINESTRING(174.76043 -...
2,606114471;Northern Mtwy;LINESTRING(174.76009 -...
3,606114472;Northern Mtwy;LINESTRING(174.76022 -...
4,605961478;Northern Mtwy;LINESTRING(174.74429 -...


In [79]:
def clearColumnByMotorway(df):  
    # Function for determining if there is the attribute of transmission 
    def addEntries(fullname):
        if fullname is "":
            return 'Entries'
        else:
            return fullname
        
    listAttrs = df['id;full_name;road_segment_wkt'].str.split(';')
    
    newdf = pd.DataFrame();
    newdf['id'] = map(lambda x: x[0], listAttrs)
    newdf['full_name'] = map(lambda x: addEntries(x[1]), listAttrs)
    newdf['road_segment_wkt'] = map(lambda x: x[2], listAttrs)
    #del df['id;full_name;road_segment_wkt']
    
    return newdf

dfNorthMotorwayInfo = clearColumnByMotorway(dfNorthMotorwayInfo)
dfNorthMotorwayInfo.to_csv("../CleanedDatasets/Northern Motorway Road Segments.csv")
dfNorthMotorwayInfo.head()

,id,full_name,road_segment_wkt
0,606114469,Northern Mtwy,"LINESTRING(174.76028 -36.80417,174.76041 -36.8..."
1,606114470,Northern Mtwy,"LINESTRING(174.76043 -36.80419,174.76053 -36.8..."
2,606114471,Northern Mtwy,"LINESTRING(174.76009 -36.80453,174.76028 -36.8..."
3,606114472,Northern Mtwy,"LINESTRING(174.76022 -36.80458,174.76043 -36.8..."
4,605961478,Northern Mtwy,"LINESTRING(174.74429 -36.77469,174.74387 -36.7..."


In [114]:
def clearColumnByCar(df):  
    listAttrs = df['id;"event_timestamp";"course_over_ground";"machine_id";"vehicle_weight_type";"speed_gps_kph";"latitude";"longitude"'].str.split(';')
    #listAttrs = df[0].str.split(';')
    newdf = pd.DataFrame();
    
    newdf['id'] = map(lambda x: x[0], listAttrs)
    newdf['event_timestamp'] = map(lambda x: x[1], listAttrs)
    newdf['course_over_ground'] = map(lambda x: x[2], listAttrs)
    newdf['machine_id'] = map(lambda x: x[3], listAttrs)
    newdf['vehicle_weight_type'] = map(lambda x: x[4], listAttrs)
    newdf['speed_gps_kph'] = map(lambda x: x[5], listAttrs)
    newdf['latitude'] = map(lambda x: x[6], listAttrs)
    newdf['longitude'] = map(lambda x: x[7], listAttrs)
    #del df['id;full_name;id;"event_timestamp";"course_over_ground";"machine_id";"vehicle_weight_type";"speed_gps_kph";"latitude";"longitude"']
    
    return newdf

mdlcopy = clearColumnByCar(mdlcopy)
#dfNorthMotorwayInfo.to_csv("../CleanedDatasets/Northern Motorway Road Segments.csv")
mdlcopy

,id,event_timestamp,course_over_ground,machine_id,vehicle_weight_type,speed_gps_kph,latitude,longitude
0,14641931007,2015-11-15 17:46:40,177,10051,"""HEAVY""",3,-36.725578,174.71482
1,14642568129,2015-11-15 18:12:41,156,10051,"""HEAVY""",13,-36.769465,174.74159
2,14641876524,2015-11-15 17:44:30,169,10051,"""HEAVY""",0,-36.723136,174.714432
3,14642262476,2015-11-15 18:00:47,150,10051,"""HEAVY""",17,-36.747435,174.723397
4,14641991113,2015-11-15 17:49:43,249,10051,"""HEAVY""",6,-36.72826,174.715083


In [92]:
import os
fileNameList = os.listdir("../Datasets/Machine data") # dir is your directory path
machineDataList = []
for filename in fileNameList:
    ##print "filename is %s " % (filename)
    dfMachineData = pd.read_csv("../Datasets/Machine data/" + filename, sep='\t')
    dfMachineData = clearColumnByCar(dfMachineData)
    dfMachineData.to_csv("../CleanedDatasets/Machine data/" + filename)
    machineDataList.append(dfMachineData)
machineDataList[0].head()

,"id;""event_timestamp"";""course_over_ground"";""machine_id"";""vehicle_weight_type"";""speed_gps_kph"";""latitude"";""longitude"""
0,"14641931007;2015-11-15 17:46:40;177;10051;""HEA..."
1,"14642568129;2015-11-15 18:12:41;156;10051;""HEA..."
2,"14641876524;2015-11-15 17:44:30;169;10051;""HEA..."
3,"14642262476;2015-11-15 18:00:47;150;10051;""HEA..."
4,"14641991113;2015-11-15 17:49:43;249;10051;""HEA..."


The codes above is to finish the data split stuff.

In [104]:
mdlcopy = machineDataList[0].head().copy()
mdlcopy

,"id;""event_timestamp"";""course_over_ground"";""machine_id"";""vehicle_weight_type"";""speed_gps_kph"";""latitude"";""longitude"""
0,"14641931007;2015-11-15 17:46:40;177;10051;""HEA..."
1,"14642568129;2015-11-15 18:12:41;156;10051;""HEA..."
2,"14641876524;2015-11-15 17:44:30;169;10051;""HEA..."
3,"14642262476;2015-11-15 18:00:47;150;10051;""HEA..."
4,"14641991113;2015-11-15 17:49:43;249;10051;""HEA..."


In [113]:
kk = mdlcopy['id;"event_timestamp";"course_over_ground";"machine_id";"vehicle_weight_type";"speed_gps_kph";"latitude";"longitude"'].str.split(';')
def pr(lists):
    print lists
df = pd.DataFrame()
df['id'] = map(lambda x: pr(x[0]), kk)
df['id']

14641931007
14642568129
14641876524
14642262476
14641991113


0    None
1    None
2    None
3    None
4    None
Name: id, dtype: object

In [114]:
def clearColumnByCar(df):  
    listAttrs = df['id;"event_timestamp";"course_over_ground";"machine_id";"vehicle_weight_type";"speed_gps_kph";"latitude";"longitude"'].str.split(';')
    #listAttrs = df[0].str.split(';')
    newdf = pd.DataFrame();
    
    newdf['id'] = map(lambda x: x[0], listAttrs)
    newdf['event_timestamp'] = map(lambda x: x[1], listAttrs)
    newdf['course_over_ground'] = map(lambda x: x[2], listAttrs)
    newdf['machine_id'] = map(lambda x: x[3], listAttrs)
    newdf['vehicle_weight_type'] = map(lambda x: x[4], listAttrs)
    newdf['speed_gps_kph'] = map(lambda x: x[5], listAttrs)
    newdf['latitude'] = map(lambda x: x[6], listAttrs)
    newdf['longitude'] = map(lambda x: x[7], listAttrs)
    #del df['id;full_name;id;"event_timestamp";"course_over_ground";"machine_id";"vehicle_weight_type";"speed_gps_kph";"latitude";"longitude"']
    
    return newdf

mdlcopy = clearColumnByCar(mdlcopy)
#dfNorthMotorwayInfo.to_csv("../CleanedDatasets/Northern Motorway Road Segments.csv")
mdlcopy

,id,event_timestamp,course_over_ground,machine_id,vehicle_weight_type,speed_gps_kph,latitude,longitude
0,14641931007,2015-11-15 17:46:40,177,10051,"""HEAVY""",3,-36.725578,174.71482
1,14642568129,2015-11-15 18:12:41,156,10051,"""HEAVY""",13,-36.769465,174.74159
2,14641876524,2015-11-15 17:44:30,169,10051,"""HEAVY""",0,-36.723136,174.714432
3,14642262476,2015-11-15 18:00:47,150,10051,"""HEAVY""",17,-36.747435,174.723397
4,14641991113,2015-11-15 17:49:43,249,10051,"""HEAVY""",6,-36.72826,174.715083
